In [1]:
import sys
!{sys.executable} -m pip install sportsipy

In [2]:
from sportsipy.nfl.boxscore import Boxscores, Boxscore
Boxscores(1,2020).games
import pandas as pd
import numpy as np
import os
import math


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split
# sklearn utilities
from sklearn import datasets
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing

# sklearn models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [3]:
def game_data_from_year(year):
    returnDF=pd.DataFrame()
    for week in range(1,22):
        weekDF=pd.DataFrame()
        for game in range(len(Boxscores(week,year).games[str(week)+"-"+str(year)])):
            weekDF=pd.concat([weekDF,Boxscore(Boxscores(week,year).games[str(week)+"-"+str(year)][game]['boxscore']).dataframe])
        weekDF["week"] = [week]*len(Boxscores(week,year).games[str(week)+"-"+str(year)])
        returnDF=pd.concat([returnDF,weekDF])
    return returnDF


In [ ]:
data200=[]
for i in range(10):
    data200.append(game_data_from_year(2000+i))
    data200[i].to_csv("data200"+str(i)+".csv")

In [ ]:
data201=[]
for i in range(10):
    data201.append(game_data_from_year(2010+i))
    data201[i].to_csv("data201"+str(i)+".csv")

In [ ]:
data200=[]
for i in range(10):
    data200.append(pd.read_csv(os.getcwd() + '/data200'+str(i)+'.csv', header=0))

In [5]:
data201=[]
for i in range(10):
    data201.append(pd.read_csv(os.getcwd() + '/data201'+str(i)+'.csv', header=0))

In [4]:
teams=['NOR', 'MIN', 'CHI', 'DET', 'MIA', 'BUF', 'TAM', 'CLE', 'PIT', 'ATL', 'OTI', 'RAI', 'NWE', 'CIN', 'HTX', 'CLT', 'JAX', 'DEN', 'NYG', 'CAR', 'CRD', 'RAM', 'SEA', 'SFO', 'GNB', 'PHI', 'WAS', 'DAL', 'RAV', 'NYJ', 'KAN', 'SDG']

In [28]:
    
newcols = ['attendance','first_downs','fourth_down_attempts','fourth_down_conversions','fumbles','fumbles_lost','interceptions',
 'net_pass_yards','pass_attempts','pass_completions','pass_touchdowns','pass_yards','penalties','points','rush_attempts',
 'rush_touchdowns','rush_yards','third_down_attempts','third_down_conversions','time_of_possession','times_sacked',
 'total_yards','turnovers','yards_from_penalties','yards_lost_from_sacks','duration','roof','surface',
 'time','temperature','humidity','wind','week','win']

def cleandata(newdata):
    teamdata = []
    for team in teams:
        teamdata.append([[],[]])

    for game in newdata:
        g=list(game)
        if g[62]=='Away':
            team1=g[1:20]+[int(g[20][0:2])+float(g[20][3:5])/60]+g[21:26]+[g[29]]+[g[56]]+g[58:60]+[g[61]]+[g[-1]]+[1]
            team2=[g[1]]+g[29:47]+[int(g[47][0:2])+float(g[47][3:5])/60]+g[48:53]+[g[29]]+[g[56]]+g[58:60]+[g[61]]+[g[-1]]+[0]



            team1[26]=  (1 if team1[26]=='Outdoors' else 0)
            team2[26]=  (1 if team2[26]=='Outdoors' else 0)
            team1[27]=  (1 if team1[27]=='Grass' else 0)
            team2[27]=  (1 if team2[27]=='Grass' else 0)
            team1[28]=  (int(team1[28].split(":")[0])+12 if team1[28].split(":")[1][2:4]=="pm" else int(team1[28].split(":")[0])) + float(team1[28].split(":")[1][0:2])/60
            team2[28]=  (int(team2[28].split(":")[0])+12 if team2[28].split(":")[1][2:4]=="pm" else int(team2[28].split(":")[0])) + float(team2[28].split(":")[1][0:2])/60

            
            
            weather1=[]
            if isinstance(team1[29],float):
                weather1=[55,0.5,9]
            elif len(team1[29].split(" "))>6:
                weather1=[int(team1[29].split(" ")[0]),float(team1[29].split(" ")[4][0:-2])/100, (0 if team1[29].split(" ")[6]=='wind,' or team1[29].split(" ")[6]=='wind' else  int(team1[29].split(" ")[6]))]
            else: 
                weather1=[int(team1[29].split(" ")[0]),.50, (0 if team1[29].split(" ")[3]=='wind,' or team1[29].split(" ")[3]=='wind' else  int(team1[29].split(" ")[3]))]


            weather2=[]
            if isinstance(team2[29],float):
                weather2=[55,0.5,9]
            elif len(team2[29].split(" "))>6:
                weather2=[int(team2[29].split(" ")[0]),float(team2[29].split(" ")[4][0:-2])/100, (0 if team2[29].split(" ")[6]=='wind,' or team2[29].split(" ")[6]=='wind' else  int(team2[29].split(" ")[6]))]
            else: 
                weather2=[int(team2[29].split(" ")[0]),.50, (0 if team2[29].split(" ")[3]=='wind,' or team2[29].split(" ")[3]=='wind' else  int(team2[29].split(" ")[3]))]



            team1=team1[0:29]+weather1+team1[30:32]
            team2=team2[0:29]+weather2+team2[30:32]

            teamdata[teams.index(g[63])][1].append(team1)
            teamdata[teams.index(g[53])][0].append(team2)
        else:
            team1=g[1:20]+[int(g[20][0:2])+float(g[20][3:5])/60]+g[21:26]+[g[29]]+[g[56]]+g[58:60]+[g[61]]+[g[-1]]+[0]
            team2=[g[1]]+g[29:47]+[int(g[47][0:2])+float(g[47][3:5])/60]+g[48:53]+[g[29]]+[g[56]]+g[58:60]+[g[61]]+[g[-1]]+[1]

            team1[26] = (1 if team1[26]=='Outdoors' else 0)
            team2[26]=  (1 if team2[26]=='Outdoors' else 0)
            team1[27]=  (1 if team1[27]=='Grass' else 0)
            team2[27]=  (1 if team2[27]=='Grass' else 0)
            team1[28]=  (int(team1[28].split(":")[0])+12 if team1[28].split(":")[1][2:4]=="pm" else int(team1[28].split(":")[0])) + float(team1[28].split(":")[1][0:2])/60
            team2[28]=  (int(team2[28].split(":")[0])+12 if team2[28].split(":")[1][2:4]=="pm" else int(team2[28].split(":")[0])) + float(team2[28].split(":")[1][0:2])/60

            
            
            weather1=[]
            if isinstance(team1[29],float):
                weather1=[55,0.5,9]
            elif len(team1[29].split(" "))>6:
                weather1=[int(team1[29].split(" ")[0]),float(team1[29].split(" ")[4][0:-2])/100, (0 if team1[29].split(" ")[6]=='wind,' or team1[29].split(" ")[6]=='wind' else  int(team1[29].split(" ")[6]))]
            else: 
                weather1=[int(team1[29].split(" ")[0]),.50, (0 if team1[29].split(" ")[3]=='wind,' or team1[29].split(" ")[3]=='wind' else  int(team1[29].split(" ")[3]))]


            weather2=[]
            if isinstance(team2[29],float):
                weather2=[55,0.5,9]
            elif len(team2[29].split(" "))>6:
                weather2=[int(team2[29].split(" ")[0]),float(team2[29].split(" ")[4][0:-2])/100, (0 if team2[29].split(" ")[6]=='wind,' or team2[29].split(" ")[6]=='wind' else  int(team2[29].split(" ")[6]))]
            else: 
                weather2=[int(team2[29].split(" ")[0]),.50, (0 if team2[29].split(" ")[3]=='wind,' or team2[29].split(" ")[3]=='wind' else  int(team2[29].split(" ")[3]))]


                
            team1=team1[0:29]+weather1+team1[30:32]
            team2=team2[0:29]+weather2+team2[30:32]





            teamdata[teams.index(g[63])][0].append(team2)
            teamdata[teams.index(g[53])][1].append(team1)
    return teamdata


def getTraining(data2014,data2015,games):
    counters=[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
    trainingX=[]
    trainingY=[]
    for game in range(len(games)):
        g=list(games[game])
        if g[62]=='Away':
            awayabbr=g[63]
            homeabbr=g[53]
            win=0
        else:
            awayabbr=g[53]
            homeabbr=g[63]
            win=1


        averagesA=[]+data2014[teams.index(awayabbr)][1][-3]
        for i in range(len(averagesA)):
            averagesA[i]+=data2014[teams.index(awayabbr)][1][-2][i]+data2014[teams.index(awayabbr)][1][-1][i]
        for i in range(counters[teams.index(awayabbr)][1]):
            for j in range(len(data2015[teams.index(awayabbr)][1][i])):
                averagesA[i]+=data2015[teams.index(awayabbr)][1][i][j]
        for i in range(len(averagesA)):
            averagesA[i] = averagesA[i]/float(counters[teams.index(awayabbr)][1]+3)



        averagesH=[]+data2014[teams.index(homeabbr)][0][-3]
        for i in range(len(averagesH)):
            averagesH[i]+=data2014[teams.index(homeabbr)][0][-2][i]+data2014[teams.index(homeabbr)][0][-1][i]
        for i in range(counters[teams.index(homeabbr)][0]):
            for j in range(len(data2015[teams.index(homeabbr)][0][i])):
                averagesH[i]+=data2015[teams.index(homeabbr)][0][i][j]
        for i in range(len(averagesH)):
            averagesH[i] = averagesH[i]/float(counters[teams.index(homeabbr)][0]+3)



        Ai=teams.index(awayabbr)
        Hi=teams.index(homeabbr)
        counters[Ai][1]+=1
        counters[Hi][0]+=1


        team1=g[1:20]+[int(g[20][0:2])+float(g[20][3:5])/60]+g[21:26]+[g[29]]+[g[56]]+g[58:60]+[g[61]]+[g[-1]]+[1]



        venue=  (1 if team1[26]=='Outdoors' else 0)
        field=  (1 if team1[27]=='Grass' else 0)
        time=  (int(team1[28].split(":")[0])+12 if team1[28].split(":")[1][2:4]=="pm" else int(team1[28].split(":")[0])) + float(team1[28].split(":")[1][0:2])/60



        weather1=[]
        if isinstance(team1[29],float):
            weather1=[55,0.5,9]
        elif len(team1[29].split(" "))>6:
            weather1=[int(team1[29].split(" ")[0]),float(team1[29].split(" ")[4][0:-2])/100, (0 if team1[29].split(" ")[6]=='wind,' or team1[29].split(" ")[6]=='wind' else  int(team1[29].split(" ")[6]))]
        else: 
            weather1=[int(team1[29].split(" ")[0]),.50, (0 if team1[29].split(" ")[3]=='wind,' or team1[29].split(" ")[3]=='wind' else  int(team1[29].split(" ")[3]))]



        trainingX.append(averagesH+averagesA+[venue]+[field]+[time]+weather1)
        trainingY.append(win)

    return trainingX,trainingY




In [34]:
xtraining=[]
ytraining=[]
for i in range(1,10):
    xtraintemp,ytraintemp=getTraining(cleandata(np.array(data201[i-1])),cleandata(np.array(data201[i])),np.array(data201[i]))
    xtraining+=xtraintemp
    ytraining+=ytraintemp

In [37]:
xtrain,xtest,ytrain,ytest= train_test_split(xtraining,ytraining)



lin = LinearRegression()
lin.fit(xtrain, ytrain)
print("lin",lin.score(xtest,ytest))
log = LogisticRegression()
log.fit(xtrain, ytrain)
print("log",log.score(xtest,ytest))
DT = DecisionTreeClassifier()
DT.fit(xtrain, ytrain)
print("DT",DT.score(xtest,ytest))
RF = RandomForestClassifier()
RF.fit(xtrain, ytrain)
print("RF",RF.score(xtest,ytest))
SV = SVC()
SV.fit(xtrain, ytrain)
print("SV",SV.score(xtest,ytest))

lin -0.011984987816265225
log 0.5341098169717138
DT 0.5058236272878536


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RF 0.5524126455906821
SV 0.5590682196339434
